In [ ]:
import gc
import time
import pymssql

import pandas as pd
import numpy as np

In [ ]:
def clean_str_to_float(val):
    if pd.isnull(val):
        return None
    
    _str = str(val).replace(" ", "").replace(",", "").replace(".", "")
    
    if not _str.isnumeric():
        print(_str, end=' ')
    
    return int(float(_str)) if  _str.isnumeric() else None

In [ ]:
try:
    connection_market.close()
except:
    pass


#conneciom MARKET
connection_market = pymssql.connect(host='10.11.12.90\BDT',
                             user='ibetlmarket',
                             port=1433,
                             password='m@rk3t2o15',
                             database='MARKET'
                            )
time.sleep(1)
gc.collect()

In [ ]:
sql_rucs_scraping = """
    SELECT * FROM ODS.MM_SUNAT_CANT_TRABAJADORES 
"""
sunat_crawler = pd.read_sql(
   sql_rucs_scraping, connection_market
).sort_values(by=['FECHA_CARGA'], ascending=False)
print(sunat_crawler.shape)

connection_market.close()
sunat_crawler.head()

In [ ]:
sunat_crawler = sunat_crawler.groupby(['PERIODO', 'RUC']).agg(
    {'NRO_TRABAJADORES': 'max', 'NRO_PENSIONISTAS': 'max', 'NRO_PRESTADORES_SERVICIO': 'max'}
).reset_index()

sunat_crawler.shape

In [ ]:
sunat_crawler.head()

In [ ]:
sunat_crawler.dtypes

### VALIDACION DE DATOS NULOS

In [ ]:
rucs_scraping = list(sunat_crawler['RUC'].unique())
print(len(rucs_scraping))

In [ ]:
print(sunat_crawler.shape)
sunat_crawler = sunat_crawler[sunat_crawler['NRO_TRABAJADORES'].notnull()]
print(sunat_crawler.shape)

In [ ]:
sunat_crawler['ANIO'] = sunat_crawler['PERIODO'].apply(lambda _: _[:4]).astype(int)
sunat_crawler['MES'] = sunat_crawler['PERIODO'].apply(lambda _: _[4:]).astype(int)
sunat_crawler = sunat_crawler.rename(columns={
    'NRO_TRABAJADORES': 'TOTAL_TRABAJADORES',
    'NRO_PENSIONISTAS': 'PENSIONISTA',
    'NRO_PRESTADORES_SERVICIO': 'PRESTADOR_SERVICIO'
})[['PERIODO', 'ANIO', 'MES', 'TOTAL_TRABAJADORES', 'PENSIONISTA', 'PRESTADOR_SERVICIO', 'RUC']]

sep = '__'
sunat_crawler['COMBINACION'] = sunat_crawler['PERIODO'] + sep +  sunat_crawler['RUC']
sunat_crawler.head()

In [ ]:
sunat_crawler = sunat_crawler[sunat_crawler['ANIO'] == 2019]
sunat_crawler.head()

In [ ]:
csv_name = "cantidad_trabajadores.csv"
cant_trab = pd.read_csv(csv_name, dtype={'RUC': str, 'PERIODO': str})

rucs_registrados = list(cant_trab['RUC'].unique())
combinaciones_registradas = list(cant_trab['COMBINACION'].unique())

print(len(rucs_registrados))
print(len(combinaciones_registradas))
print(cant_trab.shape)

cant_trab.head()

In [ ]:
sunat_crawler = sunat_crawler[
    (sunat_crawler['RUC'].isin(rucs_registrados)) & (~sunat_crawler['COMBINACION'].isin(combinaciones_registradas))
]
print(sunat_crawler.shape)

sunat_crawler['PERIODO'].value_counts().sort_index()

In [ ]:
pivoteo = pd.pivot_table(
    sunat_crawler, 
    values='TOTAL_TRABAJADORES', index=['RUC'], columns=['PERIODO'], aggfunc=np.max, fill_value=-1
).reset_index()

pivoteo.head()

In [ ]:
tiene_menos_1 = pivoteo[
    (pivoteo['201910'] != -1) & (pivoteo['201911'] == -1) &  (pivoteo['201912'] == -1)
]
tiene_menos_1

In [30]:
len(list(tiene_menos_1['RUC'].unique()))

2847

In [32]:
sunat_crawler = sunat_crawler[~sunat_crawler['RUC'].isin(list(tiene_menos_1['RUC'].unique()))]
sunat_crawler['PERIODO'].value_counts()

201912    21
201911    21
Name: PERIODO, dtype: int64

In [33]:
cant_trab.shape

(93792, 8)

In [34]:
cant_trab = pd.concat(
    [cant_trab, sunat_crawler], axis=0, ignore_index=True
)
cant_trab['PERIODO'].value_counts().sort_index()

201910    1916
201911    1939
201912    2056
2020      1031
202001    7241
202002    7241
202003    7241
202004    7241
202005    7241
202006    7241
202007    7241
202008    7241
202009    7241
202010    7241
202011    7241
202012    7241
Name: PERIODO, dtype: int64

In [42]:
cant_trab = cant_trab[cant_trab['PERIODO'] != '2020']
cant_trab.shape

(92803, 8)

In [43]:
cant_trab.shape

(92803, 8)

In [57]:
cant_trab.to_csv(csv_name, index=False)

In [51]:
cant_trab.dtypes

PERIODO               object
ANIO                   int64
MES                   object
TOTAL_TRABAJADORES    object
PENSIONISTA           object
PRESTADOR_SERVICIO    object
RUC                   object
COMBINACION           object
dtype: object

In [52]:
cant_trab['MES'] = cant_trab['MES'].astype(int)

In [53]:
cant_trab.head()

,PERIODO,ANIO,MES,TOTAL_TRABAJADORES,PENSIONISTA,PRESTADOR_SERVICIO,RUC,COMBINACION
0,202001,2020,1,172,0,24,20530611681,202001__20530611681
1,202002,2020,2,174,0,22,20530611681,202002__20530611681
2,202003,2020,3,154,0,25,20530611681,202003__20530611681
3,202004,2020,4,149,0,15,20530611681,202004__20530611681
4,202005,2020,5,146,0,19,20530611681,202005__20530611681


In [54]:
cant_trab['PERIODO'].value_counts().sort_index()

201910    1916
201911    1939
201912    2056
202001    7241
202002    7241
202003    7241
202004    7241
202005    7241
202006    7241
202007    7241
202008    7241
202009    7241
202010    7241
202011    7241
202012    7241
Name: PERIODO, dtype: int64

In [55]:
cant_trab['ANIO'].value_counts().sort_index()

2019     5911
2020    86892
Name: ANIO, dtype: int64

In [56]:
cant_trab['MES'].value_counts()

12    9297
11    9178
10    9159
9     7241
8     7241
7     7241
6     7241
5     7241
4     7241
3     7241
2     7241
1     7241
Name: MES, dtype: int64